In [2]:
import sys
assert sys.version_info >= (3, 5)
import os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import stats
import warnings
warnings.filterwarnings('ignore') # Para evitar los molestos avisos.
%matplotlib inline
from sklearn.preprocessing import StandardScaler

In [3]:
import utils as ut

In [4]:
DATA_PATH = "../data/union/End"
def load_data_prep(data_path=DATA_PATH):
    csv_path = os.path.join(data_path, "dataset_final.csv")
    return pd.read_csv(csv_path)

In [19]:
df = load_data_prep()

In [20]:
df.shape

(75, 437)

In [21]:
df1 = df[ut.filtro()]

In [22]:
df1.head()

,cod,ccbd_diameter,ccbd_iso,ccbd_mean_length,ccbd_number_of_tracts,ccbd_qa,ccbd_radius_of_end_region_1,ccbd_radius_of_end_region_2,ccbd_volume,ecpl_diameter,...,fatr_volume,ufsr_diameter,ufsr_iso,ufsr_mean_length,ufsr_number_of_tracts,ufsr_qa,ufsr_radius_of_end_region_1,ufsr_radius_of_end_region_2,ufsr_volume,label
0,sub-10171,27.3579,0.446077,111.237,36905.0,0.215172,19.1653,19.2532,65389.1,7.004120,...,19514.0,17.1792,0.503620,61.0658,4950.0,0.160238,8.82430,9.62320,14154.40,1
1,sub-10189,25.3084,0.188728,115.111,39509.0,0.089672,20.7926,24.1406,57907.5,8.893479,...,14557.9,16.8371,0.231146,52.8185,3627.0,0.071355,9.02383,10.12790,11760.10,1
2,sub-10206,26.0740,0.212524,109.945,39403.0,0.120823,16.3859,15.3304,58705.6,8.893479,...,18435.8,13.3607,0.257717,44.0673,2046.0,0.111454,8.95602,8.21973,6178.25,1
3,sub-10217,30.9647,0.464436,109.997,48988.0,0.239632,18.3277,18.7348,82833.0,10.907700,...,15469.6,15.6964,0.512934,55.8102,4247.0,0.156681,8.68760,8.48388,10799.40,1
4,sub-10225,24.6510,0.569153,131.729,44224.0,0.301554,26.1355,20.2082,62869.5,8.893479,...,13896.9,12.8347,0.683337,36.0666,742.0,0.210445,11.83780,12.60370,4666.25,1


In [23]:
fasc = [ 'cfpr_', 'ccbd_', 'ecpl_', 'ifol_', 'sl2l_', 'tral_', 'ufsl_', 'ufsr_' ]
lab = ['radius_of_end_region_1', 'radius_of_end_region_2', 'qa', 'iso']
borrar = []
for i in fasc:
    df1[i+'ip1_dif'] = df1[i+lab[2]]/(df1[i+lab[0]]+df1[i+lab[1]])
    df1[i+'ip2_norm'] = df1[i+lab[3]]/(df1[i+lab[2]])
for i in fasc:
    for j in lab:
        borrar.append(i+j)


In [25]:
df1.drop(borrar, axis=1, inplace=True)

In [26]:
df1.head()

,cod,ccbd_diameter,ccbd_mean_length,ccbd_number_of_tracts,ccbd_volume,ecpl_diameter,ecpl_mean_length,ecpl_number_of_tracts,ecpl_volume,ifol_diameter,...,ifol_ip1_dif,ifol_ip2_norm,sl2l_ip1_dif,sl2l_ip2_norm,tral_ip1_dif,tral_ip2_norm,ufsl_ip1_dif,ufsl_ip2_norm,ufsr_ip1_dif,ufsr_ip2_norm
0,sub-10171,27.3579,111.237,36905.0,65389.1,7.004120,111.475000,1048.000000,4295.100000,14.21160,...,0.007929,1.967688,0.006490,2.574465,0.006649,2.486354,0.009425,2.637159,0.008686,3.142950
1,sub-10189,25.3084,115.111,39509.0,57907.5,8.893479,100.580235,2936.541667,6774.582292,14.68170,...,0.003705,2.322339,0.002774,2.668472,0.003548,2.513058,0.003527,4.245304,0.003726,3.239371
2,sub-10206,26.0740,109.945,39403.0,58705.6,8.893479,100.580235,2936.541667,6774.582292,7.88520,...,0.009230,2.040639,0.004730,2.642847,0.004208,2.376092,0.008046,2.881791,0.006489,2.312317
3,sub-10217,30.9647,109.997,48988.0,82833.0,10.907700,95.805300,4655.000000,8952.530000,15.93850,...,0.007957,2.028316,0.007480,2.615611,0.008466,2.580978,0.006749,3.922734,0.009124,3.273747
4,sub-10225,24.6510,131.729,44224.0,62869.5,8.893479,100.580235,2936.541667,6774.582292,8.30575,...,0.021328,1.964517,0.007364,2.633665,0.008536,2.677147,0.011457,2.678544,0.008610,3.247105


In [27]:
data1 = df1[df1['label'] == 0]
#data1 = data1.drop(['cod', 'label'], axis=1).groupby(by='afsl_qa').mean()
# The correlation between all pairs of variables is easily computed with Pandas.
corr_matrix =  data1.drop(['cod', 'label'], axis=1).corr()
print(corr_matrix.shape)
corr_matrix.iloc[:10, :10]

(56, 56)


,ccbd_diameter,ccbd_mean_length,ccbd_number_of_tracts,ccbd_volume,ecpl_diameter,ecpl_mean_length,ecpl_number_of_tracts,ecpl_volume,ifol_diameter,ifol_mean_length
ccbd_diameter,1.000000,0.259609,0.842137,0.954887,0.115098,0.215174,-0.006091,0.061257,0.186846,-0.390314
ccbd_mean_length,0.259609,1.000000,0.368923,0.531758,-0.285273,-0.165056,-0.362909,-0.310253,0.176146,0.332177
ccbd_number_of_tracts,0.842137,0.368923,1.000000,0.845064,0.319564,0.418755,0.202223,0.247496,0.427156,-0.124356
ccbd_volume,0.954887,0.531758,0.845064,1.000000,0.015210,0.138474,-0.114301,-0.039773,0.210497,-0.238029
ecpl_diameter,0.115098,-0.285273,0.319564,0.015210,1.000000,0.861931,0.969923,0.985954,0.285753,-0.074137
ecpl_mean_length,0.215174,-0.165056,0.418755,0.138474,0.861931,1.000000,0.763401,0.779164,0.399982,-0.013652
ecpl_number_of_tracts,-0.006091,-0.362909,0.202223,-0.114301,0.969923,0.763401,1.000000,0.979892,0.240374,-0.043090
ecpl_volume,0.061257,-0.310253,0.247496,-0.039773,0.985954,0.779164,0.979892,1.000000,0.204685,-0.080330
ifol_diameter,0.186846,0.176146,0.427156,0.210497,0.285753,0.399982,0.240374,0.204685,1.000000,0.554670
ifol_mean_length,-0.390314,0.332177,-0.124356,-0.238029,-0.074137,-0.013652,-0.043090,-0.080330,0.554670,1.000000


In [28]:
correlation_mat = corr_matrix
corr_pairs = correlation_mat.unstack()
sorted_pairs = corr_pairs.sort_values(kind="quicksort")
sorted_pairs

ecpl_mean_length       ecpl_ip1_dif            -0.895216
ecpl_ip1_dif           ecpl_mean_length        -0.895216
tral_ip1_dif           sl2l_number_of_tracts   -0.866767
sl2l_number_of_tracts  tral_ip1_dif            -0.866767
tral_ip1_dif           tral_diameter           -0.864432
                                                  ...   
fatr_number_of_tracts  fatr_number_of_tracts    1.000000
fatr_mean_length       fatr_mean_length         1.000000
fatr_iso               fatr_iso                 1.000000
cfpr_ip1_dif           cfpr_ip1_dif             1.000000
ufsr_ip2_norm          ufsr_ip2_norm            1.000000
Length: 3136, dtype: float64

In [30]:
strong_pairs = sorted_pairs[(abs(sorted_pairs) > 0.7)]

In [31]:
print(strong_pairs.groupby(level=0).filter(lambda x: len(x)>=2).sort_values(ascending=True)[:10])

ecpl_mean_length       ecpl_ip1_dif            -0.895216
ecpl_ip1_dif           ecpl_mean_length        -0.895216
tral_ip1_dif           sl2l_number_of_tracts   -0.866767
sl2l_number_of_tracts  tral_ip1_dif            -0.866767
tral_ip1_dif           tral_diameter           -0.864432
tral_diameter          tral_ip1_dif            -0.864432
ecpl_ip1_dif           sl2l_number_of_tracts   -0.810960
sl2l_number_of_tracts  ecpl_ip1_dif            -0.810960
ecpl_ip1_dif           ecpl_diameter           -0.779510
ecpl_diameter          ecpl_ip1_dif            -0.779510
dtype: float64


In [32]:
strong_pairs.groupby(level=0).filter(lambda x: len(x)>=2).sort_values(ascending=True)[:10].index

MultiIndex([(     'ecpl_mean_length',          'ecpl_ip1_dif'),
            (         'ecpl_ip1_dif',      'ecpl_mean_length'),
            (         'tral_ip1_dif', 'sl2l_number_of_tracts'),
            ('sl2l_number_of_tracts',          'tral_ip1_dif'),
            (         'tral_ip1_dif',         'tral_diameter'),
            (        'tral_diameter',          'tral_ip1_dif'),
            (         'ecpl_ip1_dif', 'sl2l_number_of_tracts'),
            ('sl2l_number_of_tracts',          'ecpl_ip1_dif'),
            (         'ecpl_ip1_dif',         'ecpl_diameter'),
            (        'ecpl_diameter',          'ecpl_ip1_dif')],
           )

In [34]:
# Scatter plot:
labels=['ecpl_mean_length',
        'ecpl_ip1_dif', 
        'sl2l_number_of_tracts', 
        'tral_ip1_dif', 
        'tral_diameter', 
        'ecpl_diameter']
sns.set()
sns.pairplot(data1[labels],kind='reg', diag_kind='kde', plot_kws={'line_kws': {'color': 'red'}}, size = 2.5)
plt.tight_layout()
plt.show();

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().